<a target="_blank" href="https://colab.research.google.com/github/FlyingPumba/circuits-benchmark/blob/main/DEMO_InterpBench.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Code to setup InterpBench

In [1]:
!apt-get update -q && apt-get install -y --no-install-recommends libgl1-mesa-glx graphviz graphviz-dev git-lfs

zsh:1: command not found: apt-get


In [2]:
!git clone --recurse-submodules https://github.com/FlyingPumba/circuits-benchmark.git
%cd circuits-benchmark
!pip install -e .

Cloning into 'circuits-benchmark'...
remote: Enumerating objects: 4667, done.
remote: Counting objects: 100% (838/838), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 4667 (delta 579), reused 796 (delta 552), pack-reused 3829
Receiving objects: 100% (4667/4667), 19.30 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (3271/3271), done.
Submodule 'submodules/Automatic-Circuit-Discovery' (https://github.com/FlyingPumba/Automatic-Circuit-Discovery.git) registered for path 'submodules/Automatic-Circuit-Discovery'
Submodule 'submodules/iit' (https://github.com/cybershiptrooper/iit.git) registered for path 'submodules/iit'
Submodule 'submodules/tracr' (https://github.com/FlyingPumba/tracr.git) registered for path 'submodules/tracr'
Cloning into '/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/circuits-benchmark/submodules/Automatic-Circuit-Discovery'...
remote: Enumerating objects: 9802, done.        
remote: Counting objects: 100% (4036/4036), done

/Users/cybershiptrooper/src/interpretability/MATS/.iit/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/circuits-benchmark
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/FlyingPumba/auto-circuit.git to /private/var/folders/_k/_46xyqdj165bdcyw79k6758w0000gn/T/pip-install-rxied_6v/auto-circuit_17bad68c6f7443fdbdacefd732f72115
  Running command git clone --filter=blob:none --quiet https://github.com/FlyingPumba/auto-circuit.git /private/var/folders/_k/_46xyqdj165bdcyw79k6758w0000gn/T/pip-install-rxied_6v/auto-circuit_17bad68c6f7443fdbdacefd732f72115
  Resolved https://github.com/FlyingPumba/auto-circuit.git to commit 0df108df03533690d8d161529f823ba26b00af3a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

### Load model from InterpBench

In [1]:
import pickle
import torch
from transformer_lens import HookedTransformerConfig, HookedTransformer
from transformer_lens import HookedTransformer

In [2]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

task = case_3.Case3()
task_idx = task.get_index()

In [3]:
from circuits_benchmark.utils.iit.ll_model_loader import get_interp_bench_model

hl_ll_corr, model = get_interp_bench_model(task, device='cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# turn off grads
model.eval()
model.requires_grad_(False)
torch.set_grad_enabled(False)

### Run evaluations

In [5]:
# load high level model
from circuits_benchmark.utils.iit import make_iit_hl_model
import circuits_benchmark.utils.iit.correspondence as correspondence
import iit.model_pairs as mp

def make_model_pair(benchmark_case):
    hl_model = benchmark_case.build_transformer_lens_model()
    hl_model = make_iit_hl_model(hl_model, eval_mode=True)
    tracr_output = benchmark_case.get_tracr_output()
    hl_ll_corr = correspondence.TracrCorrespondence.from_output(
            case=benchmark_case, tracr_output=tracr_output
        )
    model_pair = mp.StrictIITModelPair(hl_model, model, hl_ll_corr)
    return model_pair

In [6]:
# evaluate models
import circuits_benchmark.commands.evaluation.iit.iit_eval as eval_node_effect

# model_pair = make_model_pair(task)
args = eval_node_effect.setup_args_parser(None, True)
args

Namespace(indices=None, output_dir='/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/results', device='cpu', seed=1234, weights='510', mean=1, save_to_wandb=False, batch_size=512, categorical_metric='accuracy', load_from_wandb=False, max_len=1000, same_size=False)

In [7]:
args.max_len = 100 # make this smaller or larger to include more or less datapoints 

In [8]:
model_pair = make_model_pair(task)
node_effects, eval_metrics = eval_node_effect.get_node_effects(case=task, model_pair=model_pair, args=args, use_mean_cache=True)

Moving model to device:  cpu
{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': HookPoint(), 'blocks.1.attn.hook_

100%|██████████| 5/5 [00:00<00:00, 54.18it/s]


In [9]:
node_effects

,node,status,resample_ablate_effect,mean_ablate_effect
0,"blocks.0.attn.hook_result, head 0",not_in_circuit,0.0,0.00000
1,"blocks.0.attn.hook_result, head 1",not_in_circuit,0.0,0.00000
2,"blocks.0.attn.hook_result, head 2",not_in_circuit,0.0,0.00000
3,"blocks.0.attn.hook_result, head 3",not_in_circuit,0.0,0.00000
4,"blocks.1.attn.hook_result, head 0",not_in_circuit,0.0,0.00000
5,"blocks.1.attn.hook_result, head 1",not_in_circuit,0.0,0.00000
6,"blocks.1.attn.hook_result, head 3",not_in_circuit,0.0,0.00000
7,blocks.1.mlp.hook_post,not_in_circuit,0.0,0.00000
8,blocks.0.mlp.hook_post,in_circuit,1.0,0.80000
9,"blocks.1.attn.hook_result, head 2",in_circuit,1.0,0.71375


In [10]:
print(eval_metrics)

val/iit_loss: 0.0001
val/IIA: 100.00%
val/accuracy: 100.00%
val/strict_accuracy: 100.00%


### Run ACDC

In [11]:
from circuits_benchmark.commands.evaluation.iit.acdc_utils import ACDCRunner
acdc_runner = ACDCRunner.make_default_runner(task=task_idx)
acdc_runner.args.threshold = 0.01
acdc_runner.args.output_dir = "./results/acdc_3/weight_100_100_40/threshold_0.025"
acdc_runner.args

Namespace(weights='100_100_40', output_dir='./results/acdc_3/weight_100_100_40/threshold_0.025', device='cpu', threshold=0.01, data_size=1000, using_wandb=False, load_from_wandb=False, include_mlp=False, next_token=False, use_pos_embed=False, first_cache_cpu='True', second_cache_cpu='True', zero_ablation=False, wandb_entity_name='remix_school-of-rock', wandb_group_name='default', wandb_project_name='acdc', wandb_run_name=None, wandb_dir='/tmp/wandb', wandb_mode='online', indices_mode='normal', names_mode='normal', torch_num_threads=0, max_num_epochs=100000, single_step=False, abs_value_threshold=False, images_output_dir='./results/acdc_3/weight_100_100_40/threshold_0.025/images')

In [12]:
metric_name = "kl" if model_pair.hl_model.is_categorical() else "l2"
data_size = acdc_runner.args.data_size
validation_metric = task.get_validation_metric(metric_name, model, data_size=data_size)
toks_int_values = task.get_clean_data(count=data_size).get_inputs()
toks_int_values_other = task.get_corrupted_data(count=data_size).get_inputs()
acdc_circuit, acdc_experiment = acdc_runner.run_acdc(
    tl_model=model,
    clean_dataset=toks_int_values,
    corrupt_dataset=toks_int_values_other,
    validation_metric=validation_metric,
)

Moving model to device:  cpu


dict_keys(['blocks.1.hook_resid_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_mlp_in', 'blocks.1.attn.hook_result', 'blocks.1.attn.hook_q', 'blocks.1.hook_q_input', 'blocks.1.attn.hook_k', 'blocks.1.hook_k_input', 'blocks.1.attn.hook_v', 'blocks.1.hook_v_input', 'blocks.0.hook_mlp_out', 'blocks.0.hook_mlp_in', 'blocks.0.attn.hook_result', 'blocks.0.attn.hook_q', 'blocks.0.hook_q_input', 'blocks.0.attn.hook_k', 'blocks.0.hook_k_input', 'blocks.0.attn.hook_v', 'blocks.0.hook_v_input', 'blocks.0.hook_resid_pre'])
blocks.1.hook_resid_post
blocks.1.hook_mlp_out
blocks.1.mlp.hook_post
blocks.1.mlp.hook_pre
blocks.1.hook_mlp_in
blocks.1.hook_resid_mid
blocks.1.hook_attn_out
blocks.1.attn.hook_result
blocks.1.attn.hook_z
blocks.1.attn.hook_pattern
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_v
blocks.1.attn.hook_k
blocks.1.attn.hook_q
blocks.1.hook_v_input
blocks.1.hook_k_input
blocks.1.hook_q_input
blocks.1.hook_resid_pre
blocks.0.hook_resid_post
blocks.0.hook_mlp_out
blocks.0.mlp.hook_

/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/acdc/TLACDCExperiment.py:897: UserWarning: Finished iterating
  warnings.warn("Finished iterating")


In [13]:
# load edges from InterpBench
from circuits_benchmark.utils.circuits_comparison import calculate_fpr_and_tpr, Circuit
from acdc.TLACDCCorrespondence import TLACDCCorrespondence
from circuits_benchmark.transformers.acdc_circuit_builder import build_acdc_circuit
from huggingface_hub import hf_hub_download

def get_tpr_fpr_for_acdc_circuit(acdc_circuit):
    edges_file = hf_hub_download("cybershiptrooper/InterpBench", subfolder=task_idx, filename="edges.pkl")
    edges = pickle.load(open(edges_file, "rb"))
    gt_circuit = Circuit()
    for edge in edges:
        gt_circuit.add_edge(edge[0], edge[1])

    full_corr = TLACDCCorrespondence.setup_from_model(
                model, use_pos_embed=True
            )
    full_circuit = build_acdc_circuit(full_corr)
    return calculate_fpr_and_tpr(acdc_circuit, gt_circuit, full_circuit)

get_tpr_fpr_for_acdc_circuit(acdc_circuit)

edges.pkl:   0%|          | 0.00/1.28k [00:00<?, ?B/s]


Summary:
 - Nodes TP rate: 1.0
 - Nodes FP rate: 0.0
 - Edges TP rate: 0.3333333333333333
 - Edges FP rate: 0.0


{'nodes': {'true_positive': {blocks.0.hook_mlp_out,
   blocks.1.attn.hook_result[2],
   blocks.1.hook_resid_post},
  'false_positive': set(),
  'false_negative': set(),
  'true_negative': {blocks.0.attn.hook_result[0],
   blocks.0.attn.hook_result[1],
   blocks.0.attn.hook_result[2],
   blocks.0.attn.hook_result[3],
   blocks.1.attn.hook_result[0],
   blocks.1.attn.hook_result[1],
   blocks.1.attn.hook_result[3],
   blocks.1.hook_mlp_out},
  'tpr': 1.0,
  'fpr': 0.0},
 'edges': {'true_positive': {(blocks.0.hook_mlp_out,
    blocks.1.attn.hook_result[2])},
  'false_positive': set(),
  'false_negative': {(hook_embed, blocks.0.hook_mlp_out),
   (hook_pos_embed, blocks.1.attn.hook_result[2])},
  'true_negative': {(blocks.0.attn.hook_result[0], blocks.0.hook_mlp_out),
   (blocks.0.attn.hook_result[0], blocks.1.attn.hook_result[0]),
   (blocks.0.attn.hook_result[0], blocks.1.attn.hook_result[1]),
   (blocks.0.attn.hook_result[0], blocks.1.attn.hook_result[2]),
   (blocks.0.attn.hook_result[0